In [1]:
import models
from models import model, create_cd_cluster, cd_diagram, bt_plot, glrt
from dataset_management import get_dataset, convert_to_autorank
from autorank import autorank, create_report, plot_stats
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

algorithm="algorithm";benchmark="bench_prior";seed="seed";budget="used_fidelity";loss="value"

In [12]:
data=get_dataset(dataset_name="fig5_24_at25_bad_intsteps")
# print(data.bench_prior.unique())

# Extend dataset by metafeatures
mfs=pd.read_csv("benchmark_metafeatures.csv").drop(["Name","log_int_fid","n_cond_Vars"],axis=1)
mf_names=list(mfs.columns[1:])
def add_mf(row):
    return pd.Series(mfs.loc[mfs["benchmark"]==row[benchmark].rsplit("_",1)[0]].values[0][1:])
data[["n_Vars","n_cont_Vars","n_cat_Vars","int_fid"]]=data.apply(add_mf,axis=1)


# Build the model

# Base-model
base_model = model(f"{loss}~{algorithm}", data)
print(base_model)

# Extend the model by testing which metafeatures has the biggest impact
model_list={}
for mf in mf_names:
    print(mf)
    model_list[mf] = model(f"{loss}~{algorithm}+{mf}", data)


       benchmark prior algorithm  used_fidelity     value  seed     bench_prior    regret  norm_regret  rel_rank  n_Vars  n_cont_Vars  n_cat_Vars  int_fid
0      LC-167190   bad        PB            1.0  0.521580     0   LC-167190_bad  0.980827     0.385148       3.0       7            4           0        1
1      LC-167190   bad        RS            1.0  0.224055     0   LC-167190_bad  0.223142     0.087623       2.0       7            4           0        1
2      LC-167190   bad        HB            1.0  0.216327     0   LC-167190_bad  0.203461     0.079895       1.0       7            4           0        1
3      LC-167190   bad        PB            1.0  0.521580     1   LC-167190_bad  0.980827     0.385148       3.0       7            4           0        1
4      LC-167190   bad        RS            1.0  0.234032     1   LC-167190_bad  0.248550     0.097600       1.0       7            4           0        1
...          ...   ...       ...            ...       ...   ...       

In [33]:
# for mf in mf_names:
#     glrt(model_list[mf], base_model,names=[mf, "base"])


ranked_models = sorted(list(model_list.items())+[("Base model",base_model)], key=lambda x: x[1].logLike, reverse=True)

for model_name, model_obj in ranked_models:
    print(f"Model: {model_name}")
    print(f"LogLike: {model_obj.logLike}")


Model: n_cont_Vars
LogLike: -247691.43704112462
Model: n_cat_Vars
LogLike: -247691.43704112462
Model: n_Vars
LogLike: -251270.10164641845
Model: int_fid
LogLike: -268741.6651562674
Model: Base model
LogLike: -269924.24434049905


: 